In [30]:
## Importing relevant libraries

In [31]:
from pulp import *
import pandas as pd
import numpy as np

In [8]:
## Reading resources data into a 30*30 matrix (from PabFab to 24x18PVCFlec in order)
## its been converted to its sparse form, by putting 0s for not listed resources in the input data. 

In [34]:
df_1 = pd.read_csv('Bill Of Resources.csv', index_col=0)
resources_bill = df_1.to_numpy()

In [35]:
resources_bill[29][29]

0.0

In [26]:
## Reading supply data into a 30*7 matrix 
## (Rows: from PabFab to 24x18PVCFlec in order, Columns: column[0] corresponds to week 1)

In [37]:
df_2 = pd.read_csv('supply_data.csv', index_col=0)
supply = df_2.to_numpy()

In [39]:
supply

array([[ 3000,     0,     0,     0,     0,     0,     0],
       [ 5000,     0,     0,     0,     0,     0,     0],
       [ 6500,     0,     0,     0,     0,     0,     0],
       [10500,     0,     0,     0,     0,     0,     0],
       [  200,     0,     0,     0,     0,     0,     0],
       [  400,     0,     0,     0,     0,     0,     0],
       [  400,     0,     0,     0,     0,     0,     0],
       [  180,   180,   180,   180,   150,     0,     0],
       [  240,   240,   240,   240,   240,     0,     0],
       [  240,   240,   240,   300,   300,     0,     0],
       [  480,   480,     0,   360,   360,     0,     0],
       [  240,   240,   240,   240,   240,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0],
       [    0,

In [17]:
## Reading Scrapping cost into a 30*7 matrix 
## (Rows: from PabFab to 24x18PVCFlec in order, Columns: column[0] corresponds to week 1)

In [40]:
df_3 = pd.read_csv('scr_cost.csv', index_col=0)
scr_cost = df_3.to_numpy()

In [42]:
## Reading inventory cost into a 30*7 matrix 
## (Rows: from PabFab to 24x18PVCFlec in order, Columns: column[0] corresponds to week 1)
## very high inventory cost (10000) has been modelled for the non inventorible resources. 

In [43]:
df_4 = pd.read_csv('inventory_cost.csv', index_col=0)
inv_cost = df_4.to_numpy()

In [ ]:
## Reading demand into a 30*7 matrix 
## (Rows: from PabFab to 24x18PVCFlec in order, Columns: column[0] corresponds to week 1)
## sparse metrix has been modelled for demand as 0 for items not mentioned in the data. 

In [44]:
df_5 = pd.read_csv('demand.csv', index_col=0)
demand = df_5.to_numpy()

In [ ]:
## Reading revenue into a 30*7 matrix 
## (Rows: from PabFab to 24x18PVCFlec in order, Columns: column[0] corresponds to week 1)
## sparse metrix has been modelled for demand as 0 for items not mentioned in the data. 
## it will be later modelled into a 3-D matrix assuming penalty for sending an item with delay of a week or so. 

In [45]:
df_6 = pd.read_csv('revenue.csv', index_col=0)
revenue = df_6.to_numpy()

In [46]:
penalty = 0.95

In [ ]:
## Setting up revenue array taking into account the penalty

In [58]:
revenue_3D = np.zeros((30, 7, 7))

In [59]:
revenue_3D.size

1470

In [63]:
for i in range(0,30):
    for j in range (0,7):
        for k in range (0,7):
            if(k<j):
                revenue_3D[i][j][k] = 0 
            elif (k==j):
                revenue_3D[i][j][k] = revenue[i][j]
            elif (k>j):
                revenue_3D[i][j][k] = revenue[i][j]*pow(penalty, k-j)
                

In [66]:
revenue_3D

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [ ]:
## Initiating variable arrays

In [69]:
y_i_t_tau = np.zeros((30, 7, 7)) ## units sold of product i (1D) of demand period t (2D) in period tau (3D)
x_i_t = np.zeros((30,7))   ## production of product i (row) in period t (column)
v_i_t = np.zeros((30,7))   ## scrapping of product i (row) in period t (column)
u_i_t = np.zeros((30,7))   ## inventory of product i (row) in period t (column)